### Processing SVI data and HepVu death rates data into one dataframe
This is done for all the counties in the US

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

/Users/h6x/anaconda3/envs/TDA/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#import xlsx file as a dataframe
HepVu_df = pd.read_excel('./data/HepVu_County_Opioid_Indicators_05DEC22.xlsx')

In [3]:
HepVu_df.head()

,GEO ID,State Abbreviation,County Name,Opioid Prescription Rate 2020,Narcotic Overdose Mortality Rate 2014,Narcotic Overdose Mortality Rate 2015,Narcotic Overdose Mortality Rate 2016,Narcotic Overdose Mortality Rate 2017,Narcotic Overdose Mortality Rate 2018,Narcotic Overdose Mortality Rate 2019,Narcotic Overdose Mortality Rate 2020
0,1001,AL,Autauga County,98.3,8.2,8.8,10.7,9.8,10.9,9.2,11.6
1,1003,AL,Baldwin County,65.0,18.0,20.0,16.6,15.1,14.9,14.5,27.4
2,1005,AL,Barbour County,22.8,4.4,4.5,5.7,5.8,5.2,5.7,7.6
3,1007,AL,Bibb County,24.8,17.2,16.6,22.6,21.7,23.1,19.4,27.3
4,1009,AL,Blount County,22.8,18.6,18.9,22.7,27.0,19.9,20.3,24.2


In [4]:
HepVu_df.shape

(3221, 11)

#### Updating HepVu dataframe "GEO ID" column

###### GEOID column needs to change: insert "0" infront of 4 character values to match other dataframe

In [5]:
HepVu_df["GEO ID"] = HepVu_df["GEO ID"].astype(str)

In [6]:
# Check the length of each value in the 'FIPS' column, and if it's 4 characters, prepend '0'
HepVu_df['GEO ID'] = HepVu_df['GEO ID'].apply(lambda x: x.zfill(5) if len(x) == 4 else x)

In [7]:
HepVu_df["GEO ID"][123]

'05023'

#### Updating SVI (SELECTED YEAR) dataframe
Filling the geometru colum from SVI 2020 and converting FIPS column into string+adding zero

#

In [8]:
# SVI 2018 dataset is in csv format(Issue with downloading as geo file at the moment, so using csv file for now)
SVI_df = gpd.read_file('./data/SVI data/SVI2020_US_county.gdb')

When we read from the CSV files rather than geo data file the FIPS column loose first zero value for some data. Because of that we need to turn it into string and add zero infront of those data

In [9]:
SVI_df["FIPS"] = SVI_df["FIPS"].astype(str)

In [10]:

# Check the length of each value in the 'FIPS' column, and if it's 4 characters, prepend '0'
SVI_df['FIPS'] = SVI_df['FIPS'].apply(lambda x: x.zfill(5) if len(x) == 4 else x)

Adding geometry column because CSV file of SVI 2018 does not contains it

In [11]:
# # add "geometry" column to SVI_2018_df by merging it with SVI_2020_df matching the 'FIPS' column
# SVI_2018_df = SVI_2018_df.merge(SVI_2020_df[['FIPS', 'geometry']], on='FIPS')

In [12]:
# SVI_2018_df.shape #some data is missing

Now we need to add narcotic death rates from HepVu to SVI dataframe

In [13]:
# add all the columns from HepVu_df to SVI_2020_df by matching the FIPS column to GEO ID column
SVI_HepVu_df = SVI_df.merge(HepVu_df[['GEO ID','Narcotic Overdose Mortality Rate 2020']], left_on="FIPS", right_on="GEO ID")

In [14]:
SVI_HepVu_df.head(3)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE,Shape_Length,Shape_Area,geometry,GEO ID,Narcotic Overdose Mortality Rate 2020
0,01,Alabama,AL,01001,Autauga,01001,"Autauga County, Alabama",594.455838,55639,0,...,0.1,2.0,0.7,0.2,0.3,2.052888,0.150259,"MULTIPOLYGON (((-86.92120 32.65754, -86.92035 ...",01001,11.6
1,01,Alabama,AL,01003,Baldwin,01003,"Baldwin County, Alabama",1589.835294,218289,0,...,0.1,1.9,0.4,0.3,0.2,4.278488,0.409924,"MULTIPOLYGON (((-88.02858 30.22676, -88.02399 ...",01003,27.4
2,01,Alabama,AL,01005,Barbour,01005,"Barbour County, Alabama",885.007618,25026,0,...,0.1,1.3,0.6,0.6,0.4,2.569455,0.223268,"MULTIPOLYGON (((-85.74803 31.61918, -85.74543 ...",01005,7.6


In [15]:
#get column names to a list
column_names = SVI_HepVu_df.columns.tolist()
column_names

['ST',
 'STATE',
 'ST_ABBR',
 'STCNTY',
 'COUNTY',
 'FIPS',
 'LOCATION',
 'AREA_SQMI',
 'E_TOTPOP',
 'M_TOTPOP',
 'E_HU',
 'M_HU',
 'E_HH',
 'M_HH',
 'E_POV150',
 'M_POV150',
 'E_UNEMP',
 'M_UNEMP',
 'E_HBURD',
 'M_HBURD',
 'E_NOHSDP',
 'M_NOHSDP',
 'E_UNINSUR',
 'M_UNINSUR',
 'E_AGE65',
 'M_AGE65',
 'E_AGE17',
 'M_AGE17',
 'E_DISABL',
 'M_DISABL',
 'E_SNGPNT',
 'M_SNGPNT',
 'E_LIMENG',
 'M_LIMENG',
 'E_MINRTY',
 'M_MINRTY',
 'E_MUNIT',
 'M_MUNIT',
 'E_MOBILE',
 'M_MOBILE',
 'E_CROWD',
 'M_CROWD',
 'E_NOVEH',
 'M_NOVEH',
 'E_GROUPQ',
 'M_GROUPQ',
 'EP_POV150',
 'MP_POV150',
 'EP_UNEMP',
 'MP_UNEMP',
 'EP_HBURD',
 'MP_HBURD',
 'EP_NOHSDP',
 'MP_NOHSDP',
 'EP_UNINSUR',
 'MP_UNINSUR',
 'EP_AGE65',
 'MP_AGE65',
 'EP_AGE17',
 'MP_AGE17',
 'EP_DISABL',
 'MP_DISABL',
 'EP_SNGPNT',
 'MP_SNGPNT',
 'EP_LIMENG',
 'MP_LIMENG',
 'EP_MINRTY',
 'MP_MINRTY',
 'EP_MUNIT',
 'MP_MUNIT',
 'EP_MOBILE',
 'MP_MOBILE',
 'EP_CROWD',
 'MP_CROWD',
 'EP_NOVEH',
 'MP_NOVEH',
 'EP_GROUPQ',
 'MP_GROUPQ',
 'EPL_POV15

In [16]:
# rename  'Opioid Prescription Rate 2018'
SVI_HepVu_df.rename(columns = {'Narcotic Overdose Mortality Rate 2020':'NOD_Rate'}, inplace = True)

In [17]:
# Convert the DataFrame to a GeoDataFrame
SVI_HepVu_2018_df = gpd.GeoDataFrame(SVI_HepVu_df, geometry='geometry')

In [18]:
# save SVI_HepVu_2018_df as a shapefile
SVI_HepVu_2018_df.to_file("./data/processed data/SVI 2020 with HepVu/SVI2020_US_county_with_opioid_indicators.shp")

/var/folders/j3/904q7dbj02j_jyjz7pvfgz384d40kw/T/ipykernel_2127/986067051.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  SVI_HepVu_2018_df.to_file("./data/processed data/SVI 2020 with HepVu/SVI2020_US_county_with_opioid_indicators.shp")
